问题：
多个 consumer 同时从同一个队列取任务时：
处理完成的顺序 ≠ 入队顺序

In [28]:
# 演示 多个消费者，取出的值顺序可能不一致
from queue import Queue
import threading
import time


def consumer(
    name: str, q: Queue[str | None], results: list[str], wait_for: float = 0.1
) -> None:
    while True:
        got_item = False
        task: str | None = None
        try:
            task = q.get()
            # 获取到元素后才设置状态
            got_item = True

            if task is None:
                print(f"【消费者】{name}消费任务完成")
                print()
                return

            time.sleep(wait_for)
            print(f"【消费者】{name}消费任务：{task}")
            results.append(f"{name} done:{task}")
        finally:
            # ✅ 只对“成功 get 到的 item”调用 task_done
            if got_item:
                q.task_done()


def main():
    q: Queue[str | None] = Queue()
    results: list[str] = []

    for task in ["A", "B", "C"]:
        q.put(task)

    # 多个消费者
    c1 = threading.Thread(target=consumer, args=("c1", q, results, 0.5))
    c2 = threading.Thread(target=consumer, args=("c2", q, results, 0.1))
    c3 = threading.Thread(target=consumer, args=("c3", q, results, 0.3))

    consumers = [c1, c2, c3]
    for c in consumers:
        q.put(None)

    for c in consumers:
        c.start()

    # 等待队列任务清0
    q.join()

    c1.join()
    c2.join()
    c3.join()

    print("results:", results)


main()


【消费者】c2消费任务：B
【消费者】c2消费任务完成

【消费者】c3消费任务：C
【消费者】c3消费任务完成

【消费者】c1消费任务：A
【消费者】c1消费任务完成

results: ['c2 done:B', 'c3 done:C', 'c1 done:A']


In [2]:
# 保证顺序处理任务A=>B=>C
from queue import Queue
import threading
import time
import random


def consumer(q: Queue[tuple | None], results: list[tuple]):
    while True:
        got_item = False
        task: tuple | None = None

        try:
            task = q.get()
            got_item = True
            if task is None:
                print("【消费者】消费任务完成")
                return

            time.sleep(random.uniform(0.1, 0.5))
            index, value = task
            results.append((index, f"done:{value}"))
            print(f"【消费者】消费任务：{value}")
        finally:
            if got_item:
                q.task_done()


def run_ordered_tasks(tasks: list[tuple[int, str]]):
    q = Queue(maxsize=2)
    results: list[tuple] = []

    c1 = threading.Thread(target=consumer, args=(q, results))
    c2 = threading.Thread(target=consumer, args=(q, results))

    consumers = [c1, c2]

    # 先消费，后生产，避免队列太小导致整个程序卡死
    for c in consumers:
        c.start()

    for task in tasks:
        q.put(task)
    for _ in consumers:
        q.put(None)

    q.join()
    c1.join()
    c2.join()
    return results


data = [(1, "A"), (2, "B"), (3, "C"), (4, "D"), (5, "E"), (6, "F")]
results = run_ordered_tasks(data)
results.sort(key=lambda x: x[0])

print(results)
assert [value for index, value in results] == [
    "done:A",
    "done:B",
    "done:C",
    "done:D",
    "done:E",
    "done:F",
]


【消费者】消费任务：A
【消费者】消费任务：B
【消费者】消费任务：C
【消费者】消费任务：D
【消费者】消费任务：E
【消费者】消费任务完成
【消费者】消费任务：F
【消费者】消费任务完成
[(1, 'done:A'), (2, 'done:B'), (3, 'done:C'), (4, 'done:D'), (5, 'done:E'), (6, 'done:F')]


 ## 预分配槽位
预分配槽位：并发处理乱序，但结果不用 sort，直接按 index 写回正确位置。
这一招在工程里非常常见：处理乱、提交有序，而且比 sort 更稳、更快。

三步走：预分配槽位 → 任务携带 index → worker 写回 slots[index]。
两条红线：index 从 1 开始会错位；槽位没填满必须报错。
一条习惯：先设计“结果结构”（slots），再写并发代码。


enumerate(data)
遍历成员的同时拿到一个“序号”

In [ ]:
from queue import Queue
import threading
import time
import random
from typing import Optional

Task = tuple[int, str]  # (index, value)


def consumer(q: Queue[Task | None], solts: list[str | None]):
    while True:
        got_item = False
        task: Task | None = None
        try:
            task = q.get()
            got_item = True
            if task is None:
                print("【消费者】消费任务完成")
                return
            time.sleep(random.uniform(0.1, 0.5))
            index, value = task
            solts[index] = f"done:{value}"
            print(f"【消费者】消费任务：{task}")
        finally:
            if got_item:
                q.task_done()


def run_tasks(data: list[str], maxsize: int = 2):
    q: Queue[Task | None] = Queue(maxsize=maxsize)
    solts: list[str | None] = [None] * len(data)

    t1 = threading.Thread(target=consumer, args=(q, solts))
    t2 = threading.Thread(target=consumer, args=(q, solts))
    t3 = threading.Thread(target=consumer, args=(q, solts))
    consumers = [t1, t2, t3]

    for t in consumers:
        t.start()

    for i, value in enumerate(data):
        q.put((i, value))

    for t in consumers:
        q.put(None)

    q.join()
    for t in consumers:
        t.join()
    return solts


# print(run_tasks(["a", "b", "c", "d", "e", "f"]))


def run_ordered_no_sort(data: list[str], *, maxsize: int = 3, workers: int = 2):
    q = Queue(maxsize=maxsize)
    slots: list[str | None] = [None] * len(data)
    consumer_threads = [
        threading.Thread(target=consumer, args=(q, slots)) for i in range(workers)
    ]

    for t in consumer_threads:
        t.start()

    # 生产者
    for i, value in enumerate(data):
        q.put((i, value))
    for _ in consumer_threads:  # 放 None
        q.put(None)

    q.join()

    for t in consumer_threads:
        t.join()

    # 非常重要：✅ 验收：槽位里不能再有 None
    if any(item is None for item in slots):
        raise RuntimeError("Some slots were not filled (missing results)")
    return [item for item in slots if item is not None]


out = run_ordered_no_sort(["A", "B", "C"], maxsize=2, workers=3)
assert out == ["done:A", "done:B", "done:C"]


【消费者】消费任务：(2, 'c')
【消费者】消费任务：(0, 'a')
【消费者】消费任务：(1, 'b')
【消费者】消费任务：(5, 'f')
【消费者】消费任务完成
【消费者】消费任务：(4, 'e')
【消费者】消费任务完成
【消费者】消费任务：(3, 'd')
【消费者】消费任务完成
['done:a', 'done:b', 'done:c', 'done:d', 'done:e', 'done:f']


In [ ]:
def has_empty_slot(solts: list[object | None]) -> bool:
    """判断 slots 中是否有空位"""
    return any(item is None for item in solts)
